# Demonstration Thanzi La Onse notebook

This outlines ways to use the modelling framework interactively.

In [1]:
# Import general Python modules
import numpy as np
import pandas as pd

# Import the classes we need from the tlo framework
from tlo import Date, DateOffset, Person, Simulation
from tlo.test import random_birth, random_death
from tlo.util import show_changes

In [2]:
# Create a simulation with desired modules
sim = Simulation(start_date=Date(1950, 1, 1))

rb = random_birth.RandomBirth(name='rb')
rb.pregnancy_probability = 0.1
rd = random_death.RandomDeath(name='rd')
rd.death_probability = 0.01
sim.register(rb, rd)

In [3]:
# Repeatability
sim.seed_rngs(4)

In [4]:
# Set up initial population
sim.make_initial_population(n=5)

In [5]:
# Store initial population state for comparison
initial_state = sim.population.props.copy()

In [6]:
# Run simulation
sim.simulate(end_date=Date(1951, 1, 1))

In [7]:
# Compare final & initial states
final_state = sim.population.props
show_changes(sim, initial_state, final_state)

,is_pregnant,date_of_birth,children,is_alive,date_of_death
0,True,1943-11-30 00:00:00,[],True,NaT
1,True,1944-08-31 00:00:00,[],False,1950-05-01 00:00:00
2,False,1945-10-31 00:00:00,[],False,1950-03-01 00:00:00
3,False,1940-02-29 00:00:00,[],True,NaT
4,True,1948-09-30 00:00:00,[],True,NaT
5,True,1950-11-01 00:00:00,[],True,NaT


In [8]:
# The simple display above doesn't handle 'object' type data well, e.g. the children list
for i, l in enumerate(sim.population.children):
    print('Children of person {}: {}'.format(i, l))

Children of person 0: []
Children of person 1: []
Children of person 2: []
Children of person 3: []
Children of person 4: [<Person 5>]
Children of person 5: []
